<a href="https://colab.research.google.com/github/paranthamangp/Google-Collab-Repository/blob/main/3_Send_screener_result_to_telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
'''
Fetch screener result using Python -

Technical - Data comes from a POST request.
You simply need to pick up
    (a) one cookie (ci_session - which can be done using Session object to hold cookies from initial landing page request to pass on with subsequent POST),
    (b) and one token (X-CSRF-TOKEN - which can be pulled from a meta tag in the initial request response):

Steps:
    (1) Get the screener URL and load it with Inspect element
    (2) Open Network tab & find "process"
    (3) Load Payload table and copy the scan_clause. Example below
        ( {cash} ( weekly close > weekly sma( weekly close , 21 ) and weekly close > weekly sma( weekly close , 63 ) and weekly rsi( 14 ) > 60 and 1 week ago rsi( 14 ) < 60 and market cap < 200000000000 and latest open > 200 and latest volume > 5000 ) )
    (4) Update the condition in below script

Example: Get CIS stock list from Screener and print as table
'''

# mandatory  library
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime
import requests
import pytz

#Get IST timezone
IST = pytz.timezone('Asia/Kolkata')

url = "https://chartink.com/screener/process"

# Replace 'YOUR_TELEGRAM_TOKEN' and 'YOUR_CHAT_ID' with your actual token and chat ID
#telegram_token = 'Token'
#chat_id = 'clientid'


# Get CIS stocks list query
condition = {"scan_clause": "( {cash} ( weekly close > weekly sma( weekly close , 21 ) and weekly close > weekly sma( weekly close , 63 ) and weekly rsi( 14 ) > 60 and 1 week ago rsi( 14 ) < 60 and market cap < 200000000000 and latest open > 200 and latest volume > 5000 ) )"}

# Get Cash / Nifty 500 stocks with daily RSI below 30
#condition = {"scan_clause":"( {57960} ( latest volume > latest sma( volume,10 ) * 1.5 and latest close > 200 and latest rsi( 14 ) < 30 and 1 day ago  rsi( 14 ) >= 30 ) ) "}

with requests.session() as s:
    r_data = s.get(url)
    soup = bs(r_data.content, "lxml")

    #get CSRF token from browser for Chartink Validation
    meta = soup.find("meta", {"name" : "csrf-token"})["content"]
    header = {"x-csrf-token" : meta}

    #make POST call to API
    data = s.post(url, headers=header, data=condition).json()
    screenerStockList = pd.DataFrame(data["data"])

    #Remove unwanted columns
    screenerStockList = screenerStockList.drop(columns=['sr', 'bsecode','volume'])
    screenerStockList = screenerStockList.filter(['close','nsecode','name','per_chg'])
    screenerStockList.rename(columns={'nsecode': 'StockCode', 'name': 'StockName','per_chg':'Percentage','close':'Price'}, inplace=True)

    #Print final stock list
    print("Stocks from Screener : ")
    print(screenerStockList)

    #Send result to telegram
    url = f'https://api.telegram.org/bot{telegram_token}/sendMessage'
    MessageToBeSent = 'CIS Screener result '+datetime.now(IST).strftime('%Y-%m-%d %H:%M:%S' )+ '\n' +'\n' + screenerStockList.to_string()
    payload = {
        'chat_id': chat_id,
        'text': MessageToBeSent
    }
    response = requests.post(url, json=payload)


Stocks from Screener : 
     Price StockCode                   StockName  Percentage
0  5431.15   THERMAX             Thermax Limited        4.77
1  5080.00    CRISIL              Crisil Limited       -0.08
2  1743.40  HDFCBANK           Hdfc Bank Limited       -0.36
3   656.60      DYCL          Dynamic Cables Ltd        1.08
4   533.35    SIYSIL  Siyaram Silk Mills Limited       -0.93
5   448.75   ASTERDM     Aster DM Healthcare Ltd        1.20
6   274.45       PAR   Par Drugs & Chemicals Ltd       -4.04
